<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN04/4.2-Backpropagation-ger.ipynb)

- Michael A. Nielsen, "Neural Networks and Deep Learning", Determination Press, 2015
- Souce: http://neuralnetworksanddeeplearning.com/chap2.html
- https://github.com/mnielsen



# Backpropagation - the mathematical way

Im letzten Kapitel haben wir gesehen, wie neuronale Netze ihre Gewichte und Verzerrungen mithilfe des Gradientenabstiegsalgorithmus lernen können. Es gab jedoch eine Lücke in unserer Erklärung: Wir haben nicht besprochen, **wie man den Gradienten der Kostenfunktion berechnet**. Das ist eine ziemliche Lücke! In diesem Kapitel werde ich einen schnellen Algorithmus zur Berechnung solcher Gradienten erklären, einen Algorithmus, der als Backpropagation bekannt ist.


Was wir wollen, ist ein Algorithmus, mit dem wir Gewichte und Verzerrungen so finden können, dass die Ausgabe des Netzwerks $y(x)$ für alle Trainingseingaben $x$ angenähert wird. Um zu quantifizieren, wie gut wir dieses Ziel erreichen, definieren wir eine Kostenfunktion, **manchmal auch als Verlust- oder Zielfunktion** bezeichnet:
$$
\begin{align}  C(w,b) \equiv
  \frac{1}{2n} \sum_x \| y(x) - a\|^2.
\tag{6}\end{align}
$$

Wie kann man den Gradientenabstieg zum Lernen in einem neuronalen Netz anwenden? Die Idee ist, den **Gradientenabstieg** zu verwenden, um die Gewichte $w_k$ und Offets $b_l$ zu finden, die die Kosten in Gleichung (6) minimieren. Um zu sehen, wie das funktioniert, werden wir die Aktualisierungsregel des Gradientenabstiegs neu formulieren, wobei die Gewichte und Offsets die Variablen $v_j$ ersetzen. Mit anderen Worten: Unsere "Position" hat jetzt die Komponenten $w_k$ und $b_l$, und der Gradientenvektor $\nabla C$ hat die entsprechenden Komponenten $\partial C / \partial w_k$ und $\partial C / \partial b_l$. Wenn wir die Aktualisierungsregel des Gradientenabstiegs in Form dieser Komponenten ausschreiben, haben wir

\begin{align}
  w_k & \rightarrow & w_k' = w_k-\eta \frac{\partial C}{\partial w_k} \tag{16}\\
  b_l & \rightarrow & b_l' = b_l-\eta \frac{\partial C}{\partial b_l}.
\tag{17}\end{align}



**Geschichte:**
- Der Backpropagation-Algorithmus wurde ursprünglich in den 1970er Jahren eingeführt, aber seine Bedeutung wurde erst in einem berühmten Papier [1986](http://www.nature.com/nature/journal/v323/n6088/pdf/323533a0.pdf) von [David Rumelhart](http://en.wikipedia.org/wiki/David_Rumelhart), [Geoffrey Hinton](http://www.cs.toronto.edu/~hinton/) und [Ronald Williams](http://en.wikipedia.org/wiki/Ronald_J._Williams) voll erkannt. 
- In dieser Abhandlung werden mehrere neuronale Netze beschrieben, bei denen Backpropagation viel schneller arbeitet als frühere Lernansätze, wodurch es möglich wurde, neuronale Netze zur Lösung von Problemen zu verwenden, die zuvor unlösbar waren. Heute ist der Backpropagation-Algorithmus das Arbeitspferd des Lernens in neuronalen Netzen.


## Partielle Ableitungen

Das Herzstück der Backpropagation ist ein Ausdruck für die *partielle Ableitung* 

$$\frac{\partial C}{\partial w}$$

der Kosten- (oder Verlust-) Funktion $C$ in Bezug auf ein beliebiges Gewicht $w$ (oder einen Bias $b$) im Netzwerk. Der Ausdruck sagt uns, wie schnell sich die Kosten ändern, wenn wir die Gewichte und Verzerrungen ändern. Und obwohl der Ausdruck etwas komplex ist, hat er auch etwas Schönes an sich, da jedes Element eine natürliche, intuitive Interpretation hat. Und so ist Backpropagation nicht nur ein schneller Algorithmus zum Lernen. Er gibt uns tatsächlich detaillierte Einblicke in die Art und Weise, wie das Ändern der Gewichte und Bias-Terme das Gesamtverhalten des Netzwerks verändert. Das ist es wert, im Detail studiert zu werden.

### Aufwärmen: Ein schneller matrixbasierter Ansatz zur Berechnung der Ausgabe eines neuronalen Netzwerks

Bevor wir die Backpropagation besprechen, wollen wir uns mit einem schnellen matrixbasierten Algorithmus zur Berechnung der Ausgabe eines neuronalen Netzes aufwärmen. Beginnen wir mit einer Notation, die es uns ermöglicht, auf die Gewichte im Netzwerk eindeutig zu verweisen. Wir verwenden $w_{jk}^l$, um das Gewicht für die Verbindung vom $k^{th}$ Neuron in der $(l-1)^{th}$ Schicht zum $j^{th}$ Neuron in der $l^{th}$ Schicht zu bezeichnen.

**Anmerkung: $w_{jk}^{l}$ bedeutet vom Neuron $k$ in der Schicht $(l-1)$ zum Neuron $j$ in der Schicht $l$**

Das folgende Diagramm zeigt also z. B. das Gewicht einer Verbindung vom vierten Neuron in der zweiten Schicht zum zweiten Neuron in der dritten Schicht eines Netzes:


In [ ]:
from IPython.display import Image, display
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/backprop_WeightDefinition.png"
display(Image(url=url))

Eine Eigenart der Notation ist die Anordnung der Indizes $j$ und $k$. Man könnte meinen, dass es sinnvoller ist, $j$ für das Eingangsneuron und $k$ für das Ausgangsneuron zu verwenden und nicht umgekehrt, wie es tatsächlich gemacht wird. 

Wir verwenden eine ähnliche Notation für die Biases und Aktivierungen des Netzwerks. Explizit verwenden wir $b_j^l$ für die Vorspannung des $j^{th}$ Neurons in der $l^{th}$ Schicht. Und wir verwenden $a^l_j$ für die Aktivierung des $j^{th}$-Neurons in der $l^{th}$-Schicht. Das folgende Diagramm zeigt Beispiele für die Verwendung dieser Notationen:


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/backprop_activation.png"
display(Image(url=url))

Mit diesen Notationen ist die Aktivierung $a^l_j$ des $j^{th}$ Neurons in der $l^{th}$ Schicht mit den Aktivierungen in der $(l-1)^{th}$ Schicht durch die Gleichung verbunden

$$
\begin{align} 
  a^{l}_j = \sigma\left( \sum_k w^{l}_{jk} a^{l-1}_k + b^l_j \right),
\tag{23}\end{align}
$$
wobei die Aktivierungsfunktion durch die Sigmoidfunktion gegeben ist

$$
\begin{align} 
  \sigma(z) \equiv \frac{1}{1+e^{-z}}.
\tag{3}\end{align}
$$

und wobei die Summe über alle Neuronen $k$ in der $(l-1)^{th}$ Schicht ist. 

**Vektorisierung**
- Um diesen Ausdruck in eine Matrixform umzuschreiben, definieren wir eine **Gewichtsmatrix** $w^l$ für jede Schicht $l$. 
- Die Einträge der Gewichtsmatrix $w^l$ sind einfach die Gewichte, die mit der $l^{th}$ Schicht der Neuronen verbunden sind, d.h. der Eintrag in der **$j^{th}$ Zeile und $k^{th}$ Spalte ist $w^{l}_{jk}$**. 
- Auf ähnliche Weise definieren wir für jede Schicht l einen Bias-Vektor, $b^l$. Sie können sich wahrscheinlich denken, wie das funktioniert - die Komponenten des Bias-Vektors sind einfach die Werte blj, eine Komponente für jedes Neuron in der l-ten Schicht. Und schließlich definieren wir einen Aktivierungsvektor al, dessen Komponenten die Aktivierungen $a^l_j$ sind.
- Die letzte Zutat, die wir brauchen, um (23) in eine Matrixform umzuschreiben, ist die Idee der Vektorisierung einer Funktion wie $\sigma$. Wir haben die Vektorisierung im letzten Kapitel kurz kennengelernt, aber zur Wiederholung: Die Idee ist, dass wir eine Funktion wie $\sigma$ auf jedes Element in einem Vektor $v$ anwenden wollen. Wir verwenden die offensichtliche Notation $\sigma(v)$, um diese Art der elementweisen Anwendung einer Funktion zu bezeichnen.  

Das heißt, die Komponenten von $\sigma(v)$ sind einfach $\sigma(v_j)=\sigma(v)_j$. Wenn wir zum Beispiel die Funktion $f(x)=x^2$ haben, dann hat die vektorisierte Form von $f$ den Effekt

$$
\begin{align}
  f\left(\left[ \begin{array}{c} 2 \\\ 3 \end{array} \right] \right)
  = \left[ \begin{array}{c} f(2) \\\ f(3) \end{array} \right]
  = \left[ \begin{array}{c} 4 \\\ 9 \end{array} \right],
\tag{24}\end{align}
$$

Das heisst, das vektorisierte $f$ quadriert einfach jedes Element des Vektors.

Mit diesen Notationen im Hinterkopf kann Gleichung (23) in die schöne und kompakte vektorisierte Form umgeschrieben werden

$$
\begin{align} 
  a^{l} = \sigma(w^l a^{l-1}+b^l).
\tag{25}\end{align}
$$

Mit diesem Ausdruck können wir viel globaler darüber nachdenken, wie die Aktivierungen in einer Schicht mit den Aktivierungen in der vorherigen Schicht zusammenhängen: 
- Wir wenden einfach die Gewichtsmatrix auf die Aktivierungen an, fügen dann den Bias-Vektor hinzu und wenden schließlich die Funktion $\sigma$ an

*Dieser Ausdruck ist übrigens der Grund für die bereits erwähnte Eigenart der wljk-Notation. Wenn wir j zur Indizierung des Eingangsneurons und k zur Indizierung des Ausgangsneurons verwenden würden, dann müssten wir die Gewichtsmatrix in Gleichung (25) durch die Transponierung der Gewichtsmatrix ersetzen.



Mit diesen Notationen im Hinterkopf kann Gleichung (23) in die schöne und kompakte vektorisierte Form umgeschrieben werden

$$
\begin{align} 
  a^{l} = \sigma(w^l a^{l-1}+b^l).
\tag{25}\end{align}
$$

Mit diesem Ausdruck können wir viel globaler darüber nachdenken, wie die Aktivierungen in einer Schicht mit den Aktivierungen in der vorherigen Schicht zusammenhängen: 
- Wir wenden einfach die Gewichtsmatrix auf die Aktivierungen an, fügen dann den Bias-Vektor hinzu und wenden schließlich die Funktion $\sigma$ an

*Dieser Ausdruck ist übrigens der Grund für die bereits erwähnte Eigenart der $w^l_{jk}$-Notation. Wenn wir $j$ zur Indizierung des Eingangsneurons und $k$ zur Indizierung des Ausgangsneurons verwenden würden, dann müssten wir die Gewichtsmatrix in Gleichung (25) durch die Transponierung der Gewichtsmatrix ersetzen.*


## Die beiden Annahmen, die wir über die Kostenfunktion benötigen

Das Ziel der Backpropagation ist es, die partiellen Ableitungen $\partial C / \partial w$ und ∂$\partial C / \partial b$ der Kostenfunktion $C$ in Bezug auf ein beliebiges Gewicht $w$ oder einen Bias $b$ im Netzwerk zu berechnen. Damit Backpropagation funktioniert, müssen wir zwei Hauptannahmen über die Form der Kostenfunktion machen. Bevor wir diese Annahmen aufstellen, ist es jedoch nützlich, ein Beispiel für eine Kostenfunktion im Kopf zu haben. Wir werden die quadratische Kostenfunktion aus dem letzten Kapitel verwenden (vgl. Gleichung (6)). In der Notation des letzten Abschnitts haben die quadratischen Kosten die Form

\begin{align}
  C = \frac{1}{2n} \sum_x \|y(x)-a^L(x)\|^2,
\tag{26}\end{align}

wobei: 
- $n$ die Gesamtzahl der Trainingsbeispiele ist; 
- die Summe ist über die einzelnen Trainingsbeispiele $x$; 
- $y=y(x)$ die entsprechende gewünschte Ausgabe ist; 
- $L$ bezeichnet die Anzahl der Schichten im Netz; und 
- $a^L=a^L(x)$ ist der Vektor der Aktivierungen, die vom Netz ausgegeben werden, wenn $x$ eingegeben wird.





Die **erste Annahme**, die wir brauchen, ist, dass die Kostenfunktion als Durchschnitt geschrieben werden kann

$$
C = \frac{1}{n} \sum_x C_x
$$

über Kostenfunktionen $C_x$ für einzelne Trainingsbeispiele $x$ geschrieben werden kann. Dies ist der Fall für die quadratische Kostenfunktion, bei der die Kosten für ein einzelnes Trainingsbeispiel $C_x = \frac{1}{2} \|y-a^L \|^2$ . 

Der Grund, warum wir diese Annahme brauchen, ist, dass wir mit Backpropagation die partiellen Ableitungen $\partial C_x / \partial w$ und $\partial C_x / \partial b$ für ein einzelnes Trainingsbeispiel berechnen können. Wir gewinnen dann $\partial C / \partial w$ und $\partial C / \partial b$ durch Mittelung über die Trainingsbeispiele zurück. Mit dieser Annahme im Hinterkopf nehmen wir an, dass das Trainingsbeispiel $x$ fixiert wurde, und lassen den tiefgestellten Index $x$ weg und schreiben die Kosten $C_x$ als $C$. Irgendwann werden wir das $x$ wieder einfügen, aber für den Moment ist es ein notatorisches Ärgernis, das man besser implizit lässt.

Die **zweite Annahme**, die wir über die Kosten machen, ist, dass sie als Funktion der Ausgaben des neuronalen Netzes geschrieben werden können:


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/backprop_loss.png"
display(Image(url=url))



Zum Beispiel erfüllt die quadratische Kostenfunktion diese Anforderung, da die quadratischen Kosten für ein einzelnes Trainingsbeispiel $x$ geschrieben werden können als

\begin{align}
  C = \frac{1}{2} \|y-a^L\|^2 = \frac{1}{2} \sum_j (y_j-a^L_j)^2,
\tag{27}\end{align}

und ist somit eine Funktion der Ausgangsaktivierungen. 
- Natürlich hängt diese Kostenfunktion auch von der gewünschten Ausgabe $y$ ab, und Sie fragen sich vielleicht, warum wir die Kosten nicht auch als Funktion von $y$ betrachten. 
- Erinnern Sie sich aber daran, dass das Eingabe-Trainingsbeispiel $x$ fest ist, und somit ist auch die Ausgabe $y$ ein fester Parameter. 
- Insbesondere ist es nicht etwas, das wir durch Ändern der Gewichte und Verzerrungen in irgendeiner Weise modifizieren können, d. h. es ist nicht etwas, was das neuronale Netzwerk lernt. 
- Und so macht es Sinn, $C$ als eine Funktion der Ausgangsaktivierungen $a^L$ allein zu betrachten, wobei $y$ lediglich ein Parameter ist, der hilft, diese Funktion zu definieren.

### Das Hadamard-Produkt, $s\odot t$
Der Backpropagation-Algorithmus basiert auf gängigen linearen algebraischen Operationen - Dinge wie Vektoraddition, Multiplikation eines Vektors mit einer Matrix und so weiter. Aber eine der Operationen ist etwas weniger gebräuchlich. Nehmen wir insbesondere an, dass $s$ und $t$ zwei Vektoren der gleichen Dimension sind. Dann verwenden wir $s\odot t$, um das **elementweise Produkt** der beiden Vektoren zu bezeichnen. Die Komponenten von $s\odot t$ sind also einfach $(s\odot t)_j=s_jt_j$. Als Beispiel,

\begin{align}
\left[\begin{array}{c} 1 \\\ 2 \end{array}\right] 
  \odot \left[\begin{array}{c} 3 \\\ 4 \end{array} \right]
= \left[ \begin{array}{c} 1 \cdot 3 \\\ 2 \cdot 4 \end{array} \right]
= \left[ \begin{array}{c} 3 \\\ 8 \end{array} \right].
\tag{28}\end{align}

Diese Art der elementweisen Multiplikation wird manchmal als **Hadamard-Produkt oder Schur-Produkt** bezeichnet. Wir werden sie als Hadamard-Produkt bezeichnen. Gute Matrixbibliotheken bieten in der Regel schnelle Implementierungen des Hadamard-Produkts, was bei der Implementierung von Backpropagation sehr nützlich ist.

## Die vier grundlegenden Gleichungen hinter Backpropagation

- Bei der Backpropagation geht es darum, zu verstehen, wie die Änderung der Gewichte und Verzerrungen in einem Netzwerk die Kostenfunktion verändert.
- Letztendlich bedeutet dies, die partiellen Ableitungen zu berechnen

$$
\frac{\partial C}{\partial w^l_{jk}} \quad \text{und} \quad \frac{\partial C}{ \partial b^l_j}
$$.

- Um diese zu berechnen, führen wir zunächst eine Zwischengröße ein, $\delta^l_j$, die wir den Fehler im $j^{th}$-Neuron in der $l$-ten Schicht nennen. 
- Durch Backpropagation erhalten wir eine Prozedur zur Berechnung des Fehlers $\delta^l_j$ und setzen dann $\delta^l_j$ in Beziehung zu $\frac{\partial C}{\partial w^l_{jk}}$ und $\frac{\partial C}{ \partial b^l_j}$.

Um zu verstehen, wie der Fehler definiert ist, stellen Sie sich vor, es gäbe einen Dämon in unserem neuronalen Netz:

- Der Dämon sitzt auf dem $j^{th}$ Neuron in Schicht $l$.
- Wenn die Eingabe für das Neuron eintrifft, bringt der Dämon die Operation des Neurons durcheinander. 
- Er fügt eine kleine Änderung $\Delta z^l_j$ zu der gewichteten Eingabe des Neurons hinzu, so dass das Neuron statt $\sigma(z^l_j)$ stattdessen $\sigma(z^l_j+\Delta z^l_j)$ ausgibt. 
- Diese Änderung pflanzt sich durch spätere Schichten im Netz fort und bewirkt schließlich, dass sich die Gesamtkosten um einen Betrag $\frac{\partial C}{\partial z^l_j} \Delta z^l_j$.


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/backprop_demon.png"
display(Image(url=url))

Nun ist dieser Dämon ein guter Dämon und versucht, Ihnen zu helfen, die Kosten zu verbessern, d.h. er versucht, ein $\Delta z^l_j$ zu finden, das die Kosten kleiner macht. 
- Angenommen, $\frac{\partial C}{\partial z^l_j}$ hat einen großen Wert (positiv oder negativ). Dann kann der Dämon die Kosten ziemlich stark senken, indem er $\Delta z^l_j$ so wählt, dass es das entgegengesetzte Vorzeichen zu $\frac{\partial C}{\partial z^l_j}$ hat. 
- Wenn dagegen $\frac{\partial C}{\partial z^l_j}$ nahe bei Null liegt, dann kann der Dämon die Kosten durch eine Störung der gewichteten Eingabe $\Delta z^l_j$ nicht wesentlich verbessern. Soweit der Dämon das beurteilen kann, ist das Neuron schon ziemlich nahe am Optimum.

*Das gilt natürlich nur für kleine Änderungen $\Delta z^l_j$. Wir nehmen an, dass der Dämon gezwungen ist, solche kleinen Änderungen vorzunehmen.* 

Und so gibt es einen heuristischen Sinn, in dem $\frac{\partial C}{\partial z^l_j}$ **ein Maß für den Fehler im Neuron** ist. Motiviert durch diese Geschichte, definieren wir den Fehler $\delta^l_j$ des Neurons $j$ in der Schicht $l$ durch

\begin{align} 
  \delta^l_j \equiv \frac{\partial C}{\partial z^l_j}.
\tag{29}\end{align}

Gemäß unseren üblichen Konventionen verwenden wir $\delta^l$, um den Vektor der Fehler zu bezeichnen, der zur Schicht $l$ gehört. Die Backpropagation gibt uns eine Möglichkeit, $\delta^l$ für jede Schicht zu berechnen und diese Fehler dann mit den Größen von realem Interesse, $\partial C / \partial w^l_{jk}$ und $\partial C / \partial b^l_{j}$, in Beziehung zu setzen



Sie fragen sich vielleicht, warum der Dämon die gewichtete Eingabe $z^l_j$ ändert. 
- Sicherlich wäre es natürlicher, sich vorzustellen, dass der Dämon die Ausgangsaktivierung $a^l_j$ ändert, mit dem Ergebnis, dass wir $\frac{\partial C}{\partial a^l_j}$ als unser Fehlermaß verwenden würden. 
- In der Tat, wenn Sie dies tun, funktionieren die Dinge ganz ähnlich wie in der Diskussion unten. Aber es stellt sich heraus, dass es die Darstellung der Backpropagation algebraisch ein wenig komplizierter macht. 
- Wir bleiben also bei $\delta^l_j \equiv \frac{\partial C}{\partial z^l_j}$ als Fehlermaß.

*NB: Bei Klassifikationsproblemen wie MNIST wird der Begriff "Fehler" manchmal verwendet, um die Fehlerrate bei der Klassifikation zu bezeichnen. Wenn z. B. das neuronale Netz 96,0 Prozent der Ziffern richtig klassifiziert, dann beträgt der Fehler 4,0 Prozent. Offensichtlich hat dies eine ganz andere Bedeutung als unsere $\delta$-Vektoren. In der Praxis sollten Sie keine Probleme haben zu erkennen, welche Bedeutung bei einer bestimmten Verwendung gemeint ist.



### Angriffsplan: 

Die Backpropagation basiert auf **vier fundamentalen Gleichungen**. Zusammen geben uns diese Gleichungen eine Möglichkeit, sowohl den Fehler $\delta^l$ als auch den **Gradienten der Kostenfunktion** zu berechnen. 

- Wir werden einen **kurzen Beweis** der Gleichungen geben, der hilft zu erklären, warum sie wahr sind; 
- Wir werden die Gleichungen in algorithmischer Form als "Pseudocode" wiedergeben und sehen, wie der Pseudocode als echter, laufender Python-Code implementiert werden kann; 
- und im letzten Abschnitt des Kapitels werden wir ein **intuitives Bild** davon entwickeln, was die Backpropagation-Gleichungen bedeuten und wie jemand sie von Grund auf entdecken könnte. Auf dem Weg dorthin werden wir immer wieder zu den vier grundlegenden Gleichungen zurückkehren, und wenn Sie Ihr Verständnis vertiefen, werden Ihnen diese Gleichungen vertraut und vielleicht sogar schön und natürlich erscheinen.



### BP1: Eine Gleichung für den Fehler in der Ausgabeschicht $L$, $\delta^L$: 

Die Komponenten von $\delta^L$ sind gegeben durch
\begin{align} 
  \delta^L_j = \frac{\partial C}{\partial a^L_j} \sigma'(z^L_j).
\tag{BP1}\end{align}

Dies ist ein sehr natürlicher Ausdruck. Der erste Term auf der rechten Seite, $\partial C / \partial a^L_j$, misst einfach, wie schnell sich die Kosten in Abhängigkeit von der $j^{th}$ Ausgangsaktivierung ändern. Wenn zum Beispiel $C$ nicht viel von einem bestimmten Ausgangsneuron $j$ abhängt, dann wird $\delta^L_j$ klein sein, was wir erwarten würden. Der zweite Term auf der rechten Seite, $\sigma'(z^L_j)$, misst, wie schnell sich die Aktivierungsfunktion $\sigma$ bei $z^L_j$ ändert.

Beachten Sie, dass alles in (BP1) **einfach zu berechnen ist**. Insbesondere berechnen wir $z^L_j$ während der Berechnung des Verhaltens des Netzwerks (Vorwärtspass), und es ist nur ein kleiner zusätzlicher Overhead, $\sigma'(z^L_j)$ zu berechnen. Die genaue Form von $\sigma'(z^L_j)$ hängt natürlich von der Form der Kostenfunktion ab. Vorausgesetzt, die Kostenfunktion ist bekannt, sollte es jedoch wenig Probleme bei der Berechnung von $\sigma'(z^L_j)$ geben. Wenn wir zum Beispiel die quadratische Kostenfunktion verwenden, dann

$$C = \frac{1}{2} \sum_j(y_j-a^L_j)^2$$
, und damit 

$$\frac{\partial C }{ \partial a^L_j} = (a_j^L-y_j)$$ 
was offensichtlich leicht berechenbar ist. Die Ableitung der Sigmoidfunktion kann geschrieben werden als:

$$\sigma'(z^L_j)= \sigma(z^L_j)\cdot \left( 1 - \sigma(z^L_j) \right)$$



Gleichung (BP1) ist ein komponentenweiser Ausdruck für $\delta^L$. Es ist ein absolut guter Ausdruck, aber nicht die matrixbasierte Form, die wir für die Backpropagation benötigen. Es ist jedoch einfach, die Gleichung in einer matrixbasierten Form umzuschreiben, als

\begin{align} 
  \delta^L = \nabla_a C \odot \sigma'(z^L).
\tag{BP1a}\end{align}

Hier ist $\nabla_a C$ als ein Vektor definiert, dessen Komponenten die partiellen Ableitungen $\partial C / \partial a^L_j$ sind. Man kann sich $\nabla_a C$ als Ausdruck der Änderungsrate von $C$ in Bezug auf die Ausgangsaktivierungen vorstellen. Es ist leicht zu sehen, dass die Gleichungen (BP1a) und (BP1) äquivalent sind, und aus diesem Grund werden wir von nun an (BP1) austauschbar verwenden, um auf beide Gleichungen zu verweisen. Als Beispiel haben wir im Fall der quadratischen Kosten $\nabla_a C = (a^L-y)$, und so wird die vollständig matrixbasierte Form von (BP1)

\begin{align} 
  \delta^L = (a^L-y) \odot \sigma'(z^L) = (a^L-y) \odot \sigma(z^L) \odot \left[ 1-\sigma'(z^L)\right].
\tag{30}\end{align}

Wie Sie sehen können, hat alles in diesem Ausdruck eine schöne Vektorform und lässt sich mit einer Bibliothek wie Numpy leicht berechnen.


**Beweis der Gleichung (BP1)**, die einen Ausdruck für den Ausgangsfehler, $\delta^L$, liefert. Um diese Gleichung zu beweisen, sei daran erinnert, dass per Definition

\begin{align}
  \delta^L_j = \frac{\partial C}{\partial z^L_j}.
\tag{36}\end{align}

Unter Anwendung der **Kettenregel** können wir die obige partielle Ableitung in Form von partiellen Ableitungen in Bezug auf die Ausgangsaktivierungen neu ausdrücken,
\begin{align}
  \delta^L_j = \sum_k \frac{\partial C}{\partial a^L_k} \frac{\partial a^L_k}{\partial z^L_j},
\tag{37}\end{align}

wobei die Summe über alle Neuronen $k$ in der Ausgabeschicht gilt. Natürlich hängt die Ausgangsaktivierung $a^L_k$ des $k^{th}$ Neurons nur von der gewichteten Eingabe $z^L_j$ für das $j^{th}$ Neuron ab, wenn $k=j$. Und so verschwindet $\partial a^L_k / \partial z^L_j$, wenn $k\ne j$. 



Als Ergebnis können wir die vorherige Gleichung vereinfachen zu

\begin{align}
  \delta^L_j = \frac{\partial C}{\partial a^L_j} \frac{\partial a^L_j}{\partial z^L_j}.
\tag{38}\end{align}

Wenn wir uns daran erinnern, dass $a^L_j = \sigma(z^L_j)$ ist, kann der zweite Term auf der rechten Seite als $\sigma'(z^L_j)$ geschrieben werden, und die Gleichung wird

\begin{align}
  \delta^L_j = \frac{\partial C}{\partial a^L_j} \sigma'(z^L_j),
\tag{39}\end{align}

was einfach (BP1) ist, in Komponentenform.

### BP2: Eine Gleichung für den Fehler $\delta^l$ in Bezug auf den Fehler in der nächsten Schicht, $\delta^{l+1}$

Im Einzelnen

\begin{align} 
  \delta^l = ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l),
\tag{BP2}\end{align}

wobei $(w^{l+1})^T$ die **Transponierte der Gewichtsmatrix** $w^{l+1}$ für die $(l+1)^{th}$ Schicht ist. 

Diese Gleichung erscheint kompliziert, aber jedes Element hat eine schöne Interpretation. 
- Nehmen wir an, wir kennen den Fehler $\delta^{l+1}$ auf der $(l+1)^{th}$-Schicht. 
- Wenn wir die transponierte Gewichtsmatrix, $(w^{l+1})^T$, anwenden, können wir uns dies intuitiv als **Verschiebung des Fehlers rückwärts durch das Netzwerk** vorstellen, was uns eine Art Maß für den Fehler am Ausgang der l-ten Schicht liefert. 
- Wir nehmen dann das Hadamard-Produkt $\odot \sigma'(z^l)$. 
- Dies verschiebt den Fehler rückwärts durch die Aktivierungsfunktion in der Schicht $l$ und gibt uns den Fehler $\delta^l$ in der gewichteten Eingabe der Schicht $l$.

**Beweis** der Gleichung BP2, die eine Gleichung für den Fehler $\delta^l$ in Bezug auf den Fehler in der nächsten Schicht, $δ^{l+1}, liefert.

Dazu wollen wir $\delta^l_j = \partial C / \partial z^l_j$ in Begriffe von $\delta^{l+1}_k = \partial C / \partial z^{l+1}_k$ umschreiben. Wir können dies mit Hilfe der Kettenregel tun,

\begin{align}
  \delta^l_j & = & \frac{\partial C}{\partial z^l_j} \tag{40}\\
  & = & \sum_k \frac{\partial C}{\partial z^{l+1}_k} \frac{\partial z^{l+1}_k}{\partial z^l_j} \tag{41}\\ 
  & = & \sum_k \frac{\partial z^{l+1}_k}{\partial z^l_j} \delta^{l+1}_k,
\tag{42}\end{align}

wobei wir in der letzten Zeile die beiden Terme auf der rechten Seite vertauscht und die Definition von $\delta^{l+1}_k$ ersetzt haben. Um den ersten Term in der letzten Zeile auszuwerten, beachten Sie, dass





\begin{align}
  z^{l+1}_k = \sum_j w^{l+1}_{kj} a^l_j +b^{l+1}_k = \sum_j w^{l+1}_{kj} \sigma(z^l_j) +b^{l+1}_k.
\tag{43}\end{align}

Differenziert man, erhält man
\begin{align}
  \delta^l_j = \sum_k w^{l+1}_{kj}  \delta^{l+1}_k \sigma'(z^l_j).
\tag{45}\end{align}

Zurücksubstituiert in (42) erhalten wir

\begin{align}
  \delta^l_j = \sum_k w^{l+1}_{kj}  \delta^{l+1}_k \sigma'(z^l_j).
\tag{45}\end{align}

Dies ist einfach (BP2) geschrieben in Komponentenform.

#### Propagieren des Fehlers rückwärts

Durch Kombination von (BP2) mit (BP1) können wir den Fehler $\delta^l$ für eine beliebige Schicht im Netzwerk berechnen. Wir beginnen mit (BP1), um $\delta^L$ zu berechnen, wenden dann Gleichung (BP2) an, um $\delta^{L-1}$ zu berechnen, dann wieder Gleichung (BP2), um $\delta^{L-2}$ zu berechnen, und so weiter, den ganzen Weg zurück durch das Netzwerk.



### BP3: Eine Gleichung für die Änderungsrate der Kosten in Bezug auf jede Verzerrung im Netzwerk: 

Im Besonderen:

\begin{align}  \frac{\partial C}{\partial b^l_j} =
  \delta^l_j.
\tag{BP3}\end{align}

Das heißt, der Fehler $\delta^l_j$ ist *exakt gleich* der Änderungsrate $\partial C / \partial b^l_j$. Das ist eine gute Nachricht, denn (BP1) und (BP2) haben uns bereits gesagt, wie wir $\delta^l_j$ berechnen können. Wir können (BP3) in Kurzschrift umschreiben als

\begin{align}
  \frac{\partial C}{\partial b} = \delta,
\tag{31}\end{align}

wobei davon auszugehen ist, dass $\delta$ an demselben Neuron ausgewertet wird wie der Bias $b$.



### BP4: Eine Gleichung für die Änderungsrate der Kosten in Bezug auf ein beliebiges Gewicht im Netzwerk: 

Im Besonderen:

\begin{align}
 \frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k \delta^l_j.
\tag{BP4}\end{align}

Dies sagt uns, wie wir die partiellen Ableitungen $\partial C/ \partial w^l_{jk}$ in Bezug auf die Größen $\delta^l$ und $a^{l-1}$ berechnen können, von denen wir bereits wissen, wie sie zu berechnen sind. Die Gleichung kann in einer weniger indexlastigen Notation umgeschrieben werden als
\begin{align}  \frac{\partial
    C}{\partial w} = a_{\rm in} \delta_{\rm out},
\tag{32}\end{align}


wobei es sich versteht, dass ain die Aktivierung des Neurons ist, das in das Gewicht $w$ eingegeben wird, und δout der Fehler des Neurons ist, das vom Gewicht $w$ ausgegeben wird:

Eine nette Konsequenz von Gleichung (32) ist, dass, wenn die Aktivierung $a_{\mathrm{in}}$ klein ist, $a_{\mathrm{in}}\approx 0$, der Gradiententerm $\partial C / \partial w$ ebenfalls dazu tendiert, klein zu sein. In diesem Fall sagen wir, dass das Gewicht langsam lernt, was bedeutet, dass es sich während des Gradientenabstiegs nicht viel ändert. Mit anderen Worten, eine Folge von (BP4) ist, dass Gewichte, die von Neuronen mit niedriger Aktivierung ausgegeben werden, langsam lernen.

Es gibt noch weitere Einsichten in diese Richtung, die aus (BP1)-(BP4) gewonnen werden können. Beginnen wir mit der Betrachtung der Ausgabeschicht. - Betrachten Sie den Term $\sigma'(z^L_j)$ in (BP1). 
- Erinnern Sie sich an den Graphen der Sigmoidfunktion im letzten Kapitel, dass die $\sigma$-Funktion sehr flach wird, wenn $\sigma(z^L_j)$ ungefähr 0 oder 1 ist. 
- Wenn dies der Fall ist, haben wir $\sigma'(z^L_j)\approx 0$. 
- Die Lektion ist also, dass ein Gewicht in der letzten Schicht langsam lernt, wenn das Ausgangsneuron entweder eine niedrige Aktivierung $(\approx 0)$ oder eine hohe Aktivierung $(\approx 1)$ hat. 
- In diesem Fall ist es üblich zu sagen, dass das Ausgangsneuron gesättigt ist und das *Gewicht folglich aufgehört hat zu lernen* (oder langsam lernt). Ähnliche Bemerkungen gelten auch für die Biases des Ausgangsneurons.


Wir können ähnliche Erkenntnisse für frühere Schichten gewinnen. Beachten Sie insbesondere den Term $\sigma'(z^l)$ in (BP2). Das bedeutet, dass $\delta^l_j$ wahrscheinlich klein wird, wenn das Neuron nahe der Sättigung ist. Und das wiederum bedeutet, dass alle Gewichte, die in ein gesättigtes Neuron eingegeben werden, langsam lernen. Diese Argumentation gilt nicht, wenn ${w^{l+1}}^T \delta^{l+1}$ genügend große Einträge hat, um die Kleinheit von $\sigma'(z^l_j)$ zu kompensieren. Aber ich spreche hier von der allgemeinen Tendenz.

Zusammenfassend haben wir gelernt, dass ein Gewicht langsam lernt, wenn entweder das Eingangsneuron niedrig aktiviert ist, oder wenn das Ausgangsneuron gesättigt ist, d.h. entweder hoch oder niedrig aktiviert ist.

Keine dieser Beobachtungen ist allzu sehr überraschend. Dennoch helfen sie, unser mentales Modell dessen zu verbessern, was beim Lernen eines neuronalen Netzwerks vor sich geht. Außerdem können wir diese Art der Argumentation umkehren. Es stellt sich heraus, dass die vier fundamentalen Gleichungen für jede Aktivierungsfunktion gelten, nicht nur für die Standard-Sigmoidfunktion (das liegt daran, dass, wie wir gleich sehen werden, die Beweise keine speziellen Eigenschaften von σ verwenden). Und so können wir diese Gleichungen verwenden, um Aktivierungsfunktionen zu entwerfen, die bestimmte gewünschte Lerneigenschaften haben. Als Beispiel, um Ihnen die Idee zu geben, nehmen wir an, wir würden eine (nicht-sigmoide) Aktivierungsfunktion σ so wählen, dass σ′ immer positiv ist und nie gegen Null geht. Das würde die Verlangsamung des Lernens verhindern, die auftritt, wenn gewöhnliche sigmoide Neuronen in die Sättigung gehen. Später im Buch werden wir Beispiele sehen, bei denen diese Art von Modifikation an der Aktivierungsfunktion vorgenommen wird. Wenn man sich die vier Gleichungen (BP1)-(BP4) vor Augen hält, kann man erklären, warum solche Modifikationen versucht werden und welche Auswirkungen sie haben können.

### Übung
1. Beweisen Sie die Gleichungen (BP3) und (BP4).

Damit ist der Beweis der vier Grundgleichungen der Backpropagation abgeschlossen. Der Beweis mag kompliziert erscheinen. Aber er ist eigentlich nur das Ergebnis einer sorgfältigen Anwendung der Kettenregel. Etwas weniger prägnant kann man sich die Backpropagation als einen Weg vorstellen, den Gradienten der Kostenfunktion zu berechnen, indem man systematisch die Kettenregel aus der Multivariablenrechnung anwendet. Das ist alles, was die Backpropagation wirklich ausmacht - der Rest sind Details.

# Der Backpropagation-Algorithmus

Die Backpropagation-Gleichungen bieten uns eine Möglichkeit, den Gradienten der Kostenfunktion zu berechnen. Lassen Sie uns dies explizit in Form eines Algorithmus ausschreiben:

1. **Eingabe** $x$: Setzen Sie die entsprechende Aktivierung a1 für die Eingabeschicht.
2. **Feedforward**: Für jedes $l = 2, 3, \ldots, L$ berechnen Sie 
$$
z^{l} = w^l a^{l-1}+b^l
$$ 
und 
$$
a^{l} = \sigma(z^{l})
$$.
3. **Ausgangsfehler** $\delta^L$: Berechnen Sie den Vektor

$$\delta^{L}= \nabla_a C \odot \sigma'(z^L)$$


4. **Backpropagieren Sie den Fehler**: Berechnen Sie für jedes $l = L-1, L-2, \dots, 2

$$
\delta^{l} = ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^{l})
$$.

5. **Ausgabe**: Der Gradient der Kostenfunktion ist gegeben durch


$$
\frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k \delta^l_j
$$
und 
$$
\frac{\partial C}{\partial b^l_j} = \delta^l_j
$$

Wenn Sie den Algorithmus betrachten, sehen Sie, warum er Backpropagation genannt wird. Wir berechnen die Fehlervektoren δl rückwärts, ausgehend von der letzten Schicht. Es mag sonderbar erscheinen, dass wir das Netzwerk rückwärts durchlaufen. Aber wenn Sie über den Beweis der Backpropagation nachdenken, ist die Rückwärtsbewegung eine Folge der Tatsache, dass die Kosten eine Funktion der Ausgaben des Netzwerks sind. Um zu verstehen, wie die Kosten mit früheren Gewichten und Verzerrungen variieren, müssen wir wiederholt die Kettenregel anwenden und uns rückwärts durch die Schichten arbeiten, um brauchbare Ausdrücke zu erhalten.

### Übungen
1. **Backpropagation mit einem einzelnen modifizierten Neuron:** Angenommen, wir modifizieren ein einzelnes Neuron in einem Feedforward-Netzwerk so, dass die Ausgabe des Neurons durch $f(\sum_j w_j x_j + b)$ gegeben ist, wobei $f$ eine andere Funktion als das Sigmoid ist. Wie sollte man den Backpropagation-Algorithmus in diesem Fall modifizieren?
2. **Backpropagation mit linearen Neuronen:** Angenommen, wir ersetzen die übliche nicht-lineare $\sigma$-Funktion im gesamten Netzwerk durch $\sigma(z)=z$. Schreiben Sie den Backpropagation-Algorithmus für diesen Fall neu.


In [ ]:
#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0 / (1.0 + np.exp(-z))


def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))


In [ ]:
import random

# Third-party libraries
import numpy as np


class Network(object):
    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k : k + mini_batch_size]
                for k in range(0, n, mini_batch_size)
            ]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(
                    "Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test)
                )
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [
            w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)
        ]
        self.biases = [
            b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)
        ]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x]  # list to store all the activations, layer by layer
        zs = []  # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return output_activations - y


In [ ]:
import gzip
import pickle

file = gzip.open("./data/mnist.pkl.gz", "rb")
u = pickle._Unpickler(file)
u.encoding = "latin1"
tr_d, va_d, te_d = u.load()


In [ ]:
def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


In [ ]:
training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
training_results = [vectorized_result(y) for y in tr_d[1]]
training_data = list(zip(training_inputs, training_results))
validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
validation_data = list(zip(validation_inputs, va_d[1]))
test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
test_data = list(zip(test_inputs, te_d[1]))


In [ ]:
net = Network([784, 30, 10])


In [ ]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)


# Backpropagation: das grosse Bild

Wir haben ein Bild des Fehlers entwickelt, der von der Ausgabe zurückpropagiert wird. Aber können wir noch tiefer gehen und mehr Intuition darüber entwickeln, was vor sich geht, wenn wir all diese Matrix- und Vektor-Multiplikationen durchführen? 

Es ist eine Sache, den Schritten in einem Algorithmus zu folgen, oder sogar dem Beweis zu folgen, dass der Algorithmus funktioniert. Aber das bedeutet nicht, dass Sie das Problem so gut verstehen, dass Sie den Algorithmus überhaupt erst hätten entdecken können. Gibt es eine plausible Argumentationskette, die Sie zur Entdeckung des Backpropagation-Algorithmus geführt haben könnte? In diesem Abschnitt werde ich diese beiden Rätsel angehen.

Um uns ein besseres Bild davon zu machen, was der Algorithmus macht, stellen wir uns vor, dass wir eine kleine Änderung $\Delta w^l_{jk}$ an einem Gewicht im Netzwerk, $w^l_{jk}$, vorgenommen haben:



In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/big_picture1.png"
display(Image(url=url))

Diese Änderung der Gewichtung bewirkt eine Änderung der Ausgangsaktivierung des entsprechenden Neurons:


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/big_picture2.png"
display(Image(url=url))

Das wiederum bewirkt eine Änderung in **allen** Aktivierungen in der nächsten Schicht:



In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/big_picture3.png"
display(Image(url=url))

Diese Änderungen bewirken wiederum Änderungen in der nächsten Schicht, und dann in der nächsten, und so weiter bis hin zu einer Änderung in der letzten Schicht, und dann in der Kostenfunktion:


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/big_picture4.png"
display(Image(url=url))

Die Änderung $\Delta C$ der Kosten ist mit der Änderung $\Delta w^l_{jk}$ des Gewichts durch die Gleichung verbunden

\begin{align} 
  \Delta C \approx \frac{\partial C}{\partial w^l_{jk}} \Delta w^l_{jk}.
\tag{47}\end{align}

Dies legt nahe, dass ein möglicher Ansatz zur Berechnung von $\frac{\partial C}{\partial w^l_{jk}}$ darin besteht, sorgfältig zu verfolgen, wie sich eine kleine Änderung in $w^l_{jk}$ ausbreitet, um eine kleine Änderung in $C$ zu verursachen. Wenn wir das können und dabei darauf achten, alles in leicht berechenbaren Größen auszudrücken, dann sollten wir in der Lage sein, $\frac{\partial C}{\partial w^l_{jk}}$ zu berechnen.

Versuchen wir, dies auszuführen. Die Änderung $\Delta w^l_{jk}$ bewirkt eine kleine Änderung $\Delta a^l_j$ in der Aktivierung des $j^{th}$ Neurons in der l-ten Schicht. Diese Änderung ist gegeben durch:

\begin{align} 
  \Delta a^l_j \approx \frac{\partial a^l_j}{\partial w^l_{jk}} \Delta w^l_{jk}.
\tag{48}\end{align}

Die Änderung der Aktivierung $\Delta a^l_j$ bewirkt eine Änderung aller Aktivierungen in der nächsten Schicht, d.h. der $(l+1)^{th}$ Schicht.


Wir konzentrieren uns auf die Art und Weise, wie nur eine einzige dieser Aktivierungen beeinflusst wird, sagen wir $a^{l+1}_q$,


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/big_picture5.png"
display(Image(url=url))

Tatsächlich wird es die folgende Änderung verursachen:

\begin{align}
  \Delta a^{l+1}_q \approx \frac{\partial a^{l+1}_q}{\partial a^l_j} \Delta a^l_j.
\tag{49}\end{align}

Setzt man den Ausdruck aus Gleichung (48) ein, erhält man:

\begin{align}
  \Delta a^{l+1}_q \approx \frac{\partial a^{l+1}_q}{\partial a^l_j} \frac{\partial a^l_j}{\partial w^l_{jk}} \Delta w^l_{jk}.
\tag{50}\end{align}

Natürlich wird die Änderung $\Delta a^{l+1}_q$ wiederum Änderungen in den Aktivierungen in der nächsten Schicht verursachen. In der Tat können wir uns einen Pfad durch das gesamte Netzwerk von $w^l_{jk}$ bis $C$ vorstellen, wobei jede Änderung der Aktivierung eine Änderung der nächsten Aktivierung und schließlich eine Änderung der Kosten am Ausgang bewirkt.

Wenn der Pfad durch die Aktivierungen $a^l_j, a^{l+1}_q, \ldots, a^{L-1}_n, a^L_m$ geht, dann ist der resultierende Ausdruck:

\begin{align}
  \Delta C \approx \frac{\partial C}{\partial a^L_m} 
  \frac{\partial a^L_m}{\partial a^{L-1}_n}
  \frac{\partial a^{L-1}_n}{\partial a^{L-2}_p} \ldots
  \frac{\partial a^{l+1}_q}{\partial a^l_j}
  \frac{\partial a^l_j}{\partial w^l_{jk}} \Delta w^l_{jk},
\tag{51}\end{align}

das heißt, wir haben einen Term vom Typ $\partial a / \partial a$ für jedes zusätzliche Neuron aufgenommen, das wir durchlaufen haben, sowie den Term $\partial C/\partial a^L_m$ am Ende. Dieser repräsentiert die Änderung in $C$ aufgrund von Änderungen in den Aktivierungen entlang dieses speziellen Pfades durch das Netzwerk. Natürlich gibt es viele Pfade, über die sich eine Änderung von $w^l_{jk}$ ausbreiten kann, um die Kosten zu beeinflussen, und wir haben nur einen einzigen Pfad betrachtet.  

Um die Gesamtänderung von $C$ zu berechnen, ist es plausibel, dass wir über alle möglichen Pfade zwischen dem Gewicht und den Endkosten summieren, d. h.,

\begin{align} 
  \Delta C \approx \sum_{mnp\ldots q} \frac{\partial C}{\partial a^L_m} 
  \frac{\partial a^L_m}{\partial a^{L-1}_n}
  \frac{\partial a^{L-1}_n}{\partial a^{L-2}_p} \ldots
  \frac{\partial a^{l+1}_q}{\partial a^l_j} 
  \frac{\partial a^l_j}{\partial w^l_{jk}} \Delta w^l_{jk},
\tag{52}\end{align}

wobei wir über alle möglichen Auswahlen für die Zwischenneuronen entlang des Pfades summiert haben. Im Vergleich mit (47) sehen wir, dass

\begin{align} 
  \frac{\partial C}{\partial w^l_{jk}} = \sum_{mnp\ldots q} \frac{\partial C}{\partial a^L_m} 
  \frac{\partial a^L_m}{\partial a^{L-1}_n}
  \frac{\partial a^{L-1}_n}{\partial a^{L-2}_p} \ldots
  \frac{\partial a^{l+1}_q}{\partial a^l_j} 
  \frac{\partial a^l_j}{\partial w^l_{jk}}.
\tag{53}\end{align}


Nun sieht Gleichung (53) kompliziert aus. Sie hat jedoch eine schöne intuitive Interpretation. 
- Wir berechnen die Änderungsrate von $C$ in Bezug auf ein Gewicht im Netzwerk. 
- Was uns die Gleichung sagt, ist, dass jede Kante zwischen zwei Neuronen im Netzwerk mit einem Ratenfaktor verbunden ist, der einfach die **partielle Ableitung der Aktivierung eines Neurons in Bezug auf die Aktivierung des anderen Neurons** ist.
- Die Kante vom ersten Gewicht zum ersten Neuron hat einen Ratenfaktor $\partial a^{l}_j / \partial w^l_{jk}$. 
- Der Ratenfaktor für einen Pfad ist einfach das Produkt der Ratenfaktoren entlang des Pfades. 
- Und die Gesamtänderungsrate $\partial C / \partial w^l_{jk}$ ist einfach die **Summe der Ratenfaktoren aller Pfade** vom Anfangsgewicht bis zu den Endkosten. 

Dieses Verfahren wird hier für einen einzelnen Pfad dargestellt:


In [ ]:
url = "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/master/ANN04/Bilder/big_picture6.png"
display(Image(url=url))

Was ich bis jetzt geliefert habe, ist ein heuristisches Argument, eine Art, darüber nachzudenken, was passiert, wenn man ein Gewicht in einem Netzwerk stört. 
- Zunächst könnten Sie explizite Ausdrücke für alle einzelnen partiellen Ableitungen in Gleichung (53) herleiten. Das ist mit ein wenig Kalkül leicht zu bewerkstelligen. 
- Danach könnte man versuchen, herauszufinden, wie man alle Summen über Indizes als Matrixmultiplikationen schreiben kann. Das erweist sich als mühsam und erfordert etwas Ausdauer, aber keine außergewöhnliche Einsicht. 
- Nachdem Sie all dies getan und dann so weit wie möglich vereinfacht haben, stellen Sie fest, dass Sie am Ende genau den Backpropagation-Algorithmus haben! 

**Und so können Sie sich den Backpropagation-Algorithmus als eine Möglichkeit vorstellen, die Summe über den Ratenfaktor für alle diese Pfade zu berechnen. Oder, um es etwas anders auszudrücken, der Backpropagation-Algorithmus ist eine clevere Methode, um kleine Störungen der Gewichte (und Verzerrungen) zu verfolgen, während sie sich durch das Netzwerk ausbreiten, den Ausgang erreichen und dann die Kosten beeinflussen.**


Es gibt buchstäblich Hunderte (wenn nicht Tausende) von Tutorials über Backpropagation, die
heute. Einige meiner Favoriten sind:

1. Andrew Ng’s discussion on backpropagation inside the Machine Learning course by Coursera.
2. Das stark mathematisch motivierte [Kapitel 2](http://neuralnetworksanddeeplearning.com/chap2.html) - Wie der Backpropagation-Algorithmus funktioniert aus "Neural Networks and Deep Learning" von [Michael Nielsen](https://github.com/mnielsen/neural-networks-and-deep-learning).
3. [Stanford’s cs231n](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture4.pdf) Ausführungen zur und Analyse der Backpropagation.
4. [Matt Mazur's](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/) ausgezeichnetes konkretes Beispiel (mit tatsächlich gearbeiteten Zahlen), das zeigt, wie Backpropagation funktioniert.
5. https://medium.freecodecamp.org/build-a-flexible-neural-network-with-backpropagation-in-python-acffeb7846d0
6. https://ayearofai.com/rohan-lenny-1-neural-networks-the-backpropagation-algorithm-explained-abf4609d4f9d or 
7. einige Artikel, die die Intuition des Gradientenabstiegs erklären, wie https://medium.com/datathings/neural-networks-and-backpropagation-explained-in-a-simple-way-f540a3611f5e